<a href="https://colab.research.google.com/github/Ryan-Radawiec/Projects/blob/master/happy_face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow


This File requires kt_utils and the datasets folder in CONV week 2

In [6]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


**Building a model in keras**

In [9]:
def HappyModel(input_shape):
  """
  Implementation of the happy model which determines whether a face is happy or not

  Arguments:
  input_shape -- shape of the images of the dataset

  Returns:
  model -- a Model() instance in keras
  """

  #Define the input placeholder as a tensor with shape input_shape. Think of this as your input image
  X_input = Input(input_shape)

  #Zero-padding: pads the border of X_input with zeros
  X = ZeroPadding2D((3,3))(X_input)

  #layer group1 32*32*32
  #conv-BN->ReLU block applied to X
  X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv1')(X)
  X = BatchNormalization(axis = 3, name = 'bn1')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), name ='max_pool1')(X)

  #layer group2 16*16*64
  X = ZeroPadding2D((2, 2))(X)
  #CONV -> BN -> -> ReLU Block applied to X
  X = Conv2D(64, (5, 5), strides = (1, 1), name = 'conv2')(X)
  X = BatchNormalization(axis = 3, name = 'bn2')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), name ='max_pool2')(X)

  #layer group3 8*8*128
  X = ZeroPadding2D((1, 1))(X)
  # CONV -> BN -> RELU Block applied to X
  X = Conv2D(128, (3, 3), strides = (1,1), name = 'conv3')(X)
  X = BatchNormalization(axis = 3, name ='bn3')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), name ='max_pool3')(X)

  #layer group4 8*8*64
  # CONV -> BN -> RELU Block applied to X
  X = Conv2D(64, (1, 1), strides = (1, 1), name = 'conv4')(X)
  X = BatchNormalization(axis = 3, name = 'bn4')(X)
  X = Activation('relu')(X)

  #layer group5 4*4*32
  X = ZeroPadding2D((1, 1))(X)
  # CONV -> BN -> RELU Block applied to X
  X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv5')(X)
  X = BatchNormalization(axis = 3, name = 'bn5')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), name='max_pool5')(X)
  
  # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
  X = Flatten()(X)
  X = Dense(128, activation='sigmoid', name='fc1')(X)
  X = Dense(32, activation='sigmoid', name='fc2')(X)
  X = Dense(1, activation='sigmoid', name='fc3')(X)
  
  # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
  model = Model(inputs = X_input, outputs = X, name='HappyModel')
  

  
  return model




Create the Model

In [10]:
happyModel = HappyModel((64,64,3))

Compile the model

In [11]:
happyModel.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

Train the Model

In [14]:
happyModel.fit(x=X_train, y=Y_train, epochs = 20, batch_size = 16)

Epoch 1/20
38/38 [==============================] - 0s 9ms/step - loss: 0.0081 - accuracy: 0.9967
Epoch 2/20
38/38 [==============================] - 0s 7ms/step - loss: 0.0107 - accuracy: 0.9967
Epoch 3/20
38/38 [==============================] - 0s 6ms/step - loss: 0.0044 - accuracy: 0.9983
Epoch 4/20
38/38 [==============================] - 0s 6ms/step - loss: 0.0038 - accuracy: 0.9983
Epoch 5/20
38/38 [==============================] - 0s 6ms/step - loss: 0.0030 - accuracy: 0.9983
Epoch 6/20
38/38 [==============================] - 0s 6ms/step - loss: 0.0027 - accuracy: 0.9983
Epoch 7/20
38/38 [==============================] - 0s 6ms/step - loss: 0.0094 - accuracy: 0.9967
Epoch 8/20
38/38 [==============================] - 0s 6ms/step - loss: 0.0035 - accuracy: 0.9983
Epoch 9/20
38/38 [==============================] - 0s 6ms/step - loss: 0.0023 - accuracy: 0.9983
Epoch 10/20
38/38 [==============================] - 0s 6ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 11/20
38/38 [